In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./hf_transformers/")

In [3]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import set_seed

In [4]:
from datetime import datetime
import argparse
import os

In [5]:
import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time
import csv
import numpy as np
import random


In [6]:
local = True

if local:
    storage_directory = './storage/'
    base_model = f'./{storage_directory}/checkpoint-37375'
    batch_size = 16
    codebert_address = "microsoft/codebert-base"
else:
    storage_directory = '/scratch/arminz/'
    batch_size = 64
    # base_model = f'/{storage_directory}/t5-small_global_repo-based_03-11-2021_15-28-40/checkpoint-37375/'
    base_model = f'{storage_directory}/checkpoint-37375'
    codebert_address = "/home/arminz/codebert-base"


In [7]:
exec_number = random.randint(0, 1000)

# parser = argparse.ArgumentParser()
# parser.add_argument("-a", "--append", type=int, required=True)
# parser.add_argument("-rp", "--repo_percent", type=float, required=True)
# parser.add_argument("-r", "--repo", type=str, required=True)

# args = parser.parse_args()
repo = '/data/all/data/girder/girder' #,/data/all/data/request/request'#, args.repo
repo_percent = 1.0 # args.repo_percent


In [8]:
data = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_repo_specific_final.json")
data_eslint = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_eslint_final.json")
data += data_eslint
len(data)

104804

In [9]:
data[0].repo

'/data/all/data/xpl/useless'

In [10]:
name = 'at'

In [11]:
all_warning_types = extract_warning_types(data)

# In[16]:


(repo_train_inputs, repo_train_labels, repo_val_inputs, repo_val_labels, repo_test_inputs, repo_test_labels,
 repo_train_info, repo_val_info, repo_test_info,) = create_data(data, all_warning_types, include_warning=True,
                                                                design='repo-based-included', select_repo=repo)

# In[17]:


(general_train_inputs, general_train_labels, general_val_inputs, general_val_labels, general_test_inputs,
 general_test_labels, general_train_info, general_val_info, general_test_info,) = create_data(data, all_warning_types,
                                                                                              include_warning=True,
                                                                                              design='repo-based')

splitting by : repo-based-included
train size: 37
val size: 14
test size: 52
splitting by : repo-based
train size: 75087
val size: 25055
test size: 2383


In [12]:
samples = int(repo_percent * len(repo_train_inputs))
print(f'all repo samples: {len(repo_train_inputs)}\ntrain samples of repo: {samples} ({repo_percent})')

all repo samples: 37
train samples of repo: 37 (1.0)


In [13]:
# added_inputs_train += repo_train_inputs[:samples]
# added_labels_train += repo_train_labels[:samples]
# added_info_train += repo_train_info[:samples]


In [14]:
# added_inputs_val += repo_val_inputs
# added_labels_val += repo_val_labels
# added_info_val += repo_val_info

In [15]:
tokenizer = T5Tokenizer.from_pretrained(base_model)

# In[48]:


train_dataset = create_dataset(repo_train_inputs[:samples], repo_train_labels[:samples], tokenizer, pad_truncate=True, max_length=128)
val_dataset = create_dataset(repo_val_inputs, repo_val_labels, tokenizer, pad_truncate=True)
# test_dataset = create_dataset(repo_val_inputs, repo_val_labels, tokenizer, pad_truncate=True)
# 
# In[49]:


now = datetime.now()
test_result_directory = f'{storage_directory}/fine-tune-result'
full_name = f'{name}_{exec_number}_{repo.rsplit("/", 1)[1][-20:]}_{repo_percent}_{samples}'
model_directory = f'{storage_directory}/tmp/{full_name}'
model_directory

/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


'./storage//tmp/at_122_girder_1.0_37'

In [16]:
lr = 4e-3
ws = 300
wd = 0.4

In [17]:
tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model)
model.resize_token_embeddings(len(tokenizer))

Embedding(32104, 512)

In [18]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32104, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32104, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [33]:
model.encoder.block[1]

T5Block(
  (layer): ModuleList(
    (0): T5LayerSelfAttention(
      (SelfAttention): T5Attention(
        (q): Linear(in_features=512, out_features=512, bias=False)
        (k): Linear(in_features=512, out_features=512, bias=False)
        (v): Linear(in_features=512, out_features=512, bias=False)
        (o): Linear(in_features=512, out_features=512, bias=False)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): T5LayerFF(
      (DenseReluDense): T5DenseReluDense(
        (wi): Linear(in_features=512, out_features=2048, bias=False)
        (wo): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [19]:
for item in model._modules:
    print(item)

shared
encoder
decoder
lm_head


In [20]:
for item in model.encoder._modules:
    print(item)

embed_tokens
block
final_layer_norm
dropout


In [21]:
# model.encoder.block.append()

In [22]:
# [item.requires_grad for item in model.parameters()]

In [23]:
# for item in model.parameters():
#     item.requires_grad = True
    
for item in model.shared.parameters():
    item.requires_grad = False

for item in model.encoder.embed_tokens.parameters():
    item.requires_grad = False
    
for item in model.encoder.block[:2].parameters():
    item.requires_grad = False
    
for item in model.decoder.embed_tokens.parameters():
    item.requires_grad = False
    
for item in model.decoder.block[:2].parameters():
    item.requires_grad = False
    
        
# # # for item in model.lm_head.parameters():
# # #     item.requires_grad = True

In [24]:
from transformers import EarlyStoppingCallback

training_args = Seq2SeqTrainingArguments(
    output_dir=model_directory,
    num_train_epochs=70,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=ws,
    weight_decay=wd,
    logging_dir=model_directory,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=lr,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    eval_accumulation_steps=1,  # set this lower, if testing or validation crashes
    disable_tqdm=False,
    predict_with_generate=True,  # never set this to false.
    seed=42,  # default value
)


In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    #     compute_metrics=compute_metrics
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.061470,0.033200,422.175000
2,No log,0.058848,0.033000,423.786000
3,No log,0.054939,0.033100,423.432000
4,No log,0.050605,0.033100,422.591000
5,No log,0.046551,0.033100,423.453000
6,No log,0.043464,0.033600,416.533000
7,No log,0.041667,0.032900,425.802000
8,No log,0.040434,0.038900,359.462000
9,No log,0.039450,0.033000,424.580000
10,No log,0.038783,0.033200,422.165000


TrainOutput(global_step=69, training_loss=0.05268378879712975, metrics={'train_runtime': 23.8192, 'train_samples_per_second': 8.816, 'total_flos': 37374973633536, 'epoch': 23.0})

In [27]:
print('eval', trainer.evaluate()['eval_loss'])

eval 0.03341660276055336


In [28]:
best_model_dir = f'{model_directory}/best/'
trainer.save_model(best_model_dir)
print('best model dir', best_model_dir)

best model dir ./storage//tmp/at_87_girder_1.0_37/best/


In [29]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [30]:

os.system(
    f'python hf_transformers/tfix_testing.py --load-model {best_model_dir} -bs {batch_size} --model-name t5-small -d repo-based-included -r {repo}')

/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
6it [00:05,  1.10it/s]                       

start time:  18:45:10
['no-invalid-this', 'no-throw-literal', 'no-new-wrappers', 'guard-for-in', 'no-new-object', 'comma-style', 'prefer-spread', 'no-caller', 'no-extra-bind', 'no-array-constructor', 'prefer-rest-params', 'generator-star-spacing', 'no-this-before-super', 'no-extend-native', 'no-undef', 'no-useless-escape', 'no-dupe-keys', 'no-console', 'no-constant-condition', 'no-duplicate-case', 'no-empty', 'no-extra-semi', 'no-redeclare', 'no-cond-assign', 'no-extra-boolean-cast', 'no-fallthrough', 'no-unreachable', 'valid-typeof', 'no-unsafe-finally', 'no-unused-vars', 'no-debugger', 'no-unsafe-negation', 'no-case-declarations', 'no-self-assign', 'no-process-exit', 'no-inner-declarations', 'for-direction', 'no-compare-neg-zero', 'no-sparse-arrays', 'no-func-assign', 'no-const-assign', 'no-global-assign', 'use-isnan', 'no-unused-labels', 'require-yield', 'getter-return', 'no-dupe-class-members', 'no-ex-assign', 'constructor-super', 'no-new-symbol', 'no-empty-pattern', 'no-class-assi

0

In [31]:
import shutil

In [32]:
shutil.rmtree(model_directory)